In [75]:
import pandas as pd
import os
import requests
import bs4 as bs
import nltk
import re
from nltk.corpus import words, stopwords
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize, sent_tokenize, pos_tag
#from gensim import corpora
import spacy
#spacy.load('en')
from spacy.lang.en import English

In [23]:
urls = ['https://www.sec.gov/Archives/edgar/data/320193/000032019318000070/a10-qq220183312018.htm',
       'https://www.sec.gov/Archives/edgar/data/6201/000000620118000016/aalq110-q033118.htm',
       'https://www.sec.gov/Archives/edgar/data/8177/000114036118022936/form10q.htm',
       'https://www.sec.gov/Archives/edgar/data/1158114/000155837018004322/aaoi-20180331x10q.htm',
       'https://www.sec.gov/Archives/edgar/data/824142/000082414216000225/aaon10-qq32016.htm',
       'https://www.sec.gov/Archives/edgar/data/881890/000114036117041851/form10q.htm',
       'https://www.sec.gov/Archives/edgar/data/351569/000035156918000007/abcb03-31x201810xq.htm',
       'https://www.sec.gov/Archives/edgar/data/1466815/000146681518000038/abcd-33118x10q.htm',
       'https://www.sec.gov/Archives/edgar/data/1157377/000115737717000016/abco-2017930x10q.htm',
       'https://www.sec.gov/Archives/edgar/data/894405/000155837017008566/arcb-20170930x10q.htm',
        'https://www.sec.gov/Archives/edgar/data/1409012/000119312510179509/d10q.htm',
        'https://www.sec.gov/Archives/edgar/data/874501/000087450118000065/a02-014ambcx20180331x10q.htm',
        'https://www.sec.gov/Archives/edgar/data/815094/000156459016022298/abmd-10q_20160630.htm',
        'https://www.sec.gov/Archives/edgar/data/1070494/000156459017022271/acad-10q_20170930.htm',
        'https://www.sec.gov/Archives/edgar/data/817473/000081747316000163/acas10q93016.htm'        
       ]

In [20]:
url = 'https://www.sec.gov/Archives/edgar/data/6201/000000620118000016/aalq110-q033118.htm'

In [3]:
def get_texts_from_url(url):
    main_page = str(requests.get(url).text)
    page_soup = bs.BeautifulSoup(main_page, 'html.parser')
    divs = page_soup.find_all('div')
    all_text = []
    for div in divs:
        text = div.getText()
        if len(text) > 200 and text[0] != '\xa0':
            all_text.append(text)
    return all_text

In [61]:
def combine_text(text):
    file = ""
    for i in range(len(text)):
        ## removes all the /xa0 /n /t garbage and some extra spaces
        cleaner_text = re.sub('\s+', ' ', text[i]).strip()
        file = file + " " + cleaner_text
        
    return file

In [122]:
en_stop = nltk.corpus.stopwords.words('english')
parser = English()

def tokenize(text):
    '''
    Separates each word from the others, making them lower case and removing URLs and possible screen names that show up.
    '''

    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def prepare_text(text):
    '''
    Tokenizes the text, removes short words, removes stop words, and then gets the lemma of each word.
    '''

    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    
    pos_tags = nltk.pos_tag(tokens)
    tokens = [pos_tag[0] for pos_tag in pos_tags if pos_tag[1][0] == "N"]
    
    ## do we want lemma?
    ##tokens = [get_lemma(token) for token in tokens]
    return tokens

In [125]:
def get_all_files(urls):
    all_files = []
    for url in urls:
        text = get_texts_from_url(url)
        file = combine_text(text)
        
        tokenized = prepare_text(file)
        
        all_files.append(tokenized)
        
    return all_files

In [126]:
all_files = get_all_files(urls)

In [127]:
all_files[0]

['california',
 'jurisdictionof',
 'incorporation',
 'organization',
 'i.r.s.',
 'employer',
 'identification',
 'apple',
 'waycupertino',
 'executive',
 'offices',
 'jurisdictionof',
 'incorporation',
 'organization',
 'i.r.s.',
 'employer',
 'identification',
 'apple',
 'waycupertino',
 'executive',
 'offices',
 'registrant',
 'reports',
 'section',
 'securities',
 'months',
 'period',
 'registrant',
 'reports',
 'filing',
 'requirements',
 'registrant',
 'interactive',
 'regulation',
 'chapter',
 'months',
 'period',
 'registrant',
 'submit',
 'files',
 'filer',
 'filer',
 'reporting',
 'company',
 'growth',
 'company',
 'filer',
 'company',
 'growth',
 'company',
 'exchange',
 'growth',
 'company',
 'check',
 'registrant',
 'transition',
 'period',
 'accounting',
 'standards',
 'section',
 'exchange',
 'apple',
 'subsidiaries',
 'apple',
 'company',
 'manufactures',
 'markets',
 'communication',
 'media',
 'devices',
 'computers',
 'variety',
 'software',
 'services',
 'accessories